![](2023-07-24-10-52-10.png)

In [ ]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install langchainhub
!pip install chromadb
!pip install pypdf
!pip install tiktoken
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

# Building a Simple QA System for Chatting with a PDF

This part of the training will be mostly hands on with the code for building the qa PDF system with langchain.

In [19]:
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain

In [20]:
pdf_path = "./assets-resources/llm_paper_know_dont_know.pdf"
loader = PyPDFLoader(pdf_path) # LOAD

In [21]:
pdf_docs = loader.load_and_split() # SPLIT
pdf_docs

[Document(page_content='Do Large Language Models Know What They Don’t Know?\nZhangyue Yin♢Qiushi Sun♠Qipeng Guo♢\nJiawen Wu♢Xipeng Qiu♢∗Xuanjing Huang♢\n♢School of Computer Science, Fudan University\n♠Department of Mathematics, National University of Singapore\n{yinzy21,jwwu21}@m.fudan.edu.cn qiushisun@u.nus.edu\n{qpguo16,xpqiu,xjhuang}@fudan.edu.cn\nAbstract\nLarge language models (LLMs) have a wealth\nof knowledge that allows them to excel in vari-\nous Natural Language Processing (NLP) tasks.\nCurrent research focuses on enhancing their\nperformance within their existing knowledge.\nDespite their vast knowledge, LLMs are still\nlimited by the amount of information they can\naccommodate and comprehend. Therefore, the\nability to understand their own limitations on\nthe unknows, referred to as self-knowledge,\nis of paramount importance. This study aims\nto evaluate LLMs’ self-knowledge by assess-\ning their ability to identify unanswerable or\nunknowable questions. We introduce an au

In [22]:
doc_obj = pdf_docs[0]
doc_obj

Document(page_content='Do Large Language Models Know What They Don’t Know?\nZhangyue Yin♢Qiushi Sun♠Qipeng Guo♢\nJiawen Wu♢Xipeng Qiu♢∗Xuanjing Huang♢\n♢School of Computer Science, Fudan University\n♠Department of Mathematics, National University of Singapore\n{yinzy21,jwwu21}@m.fudan.edu.cn qiushisun@u.nus.edu\n{qpguo16,xpqiu,xjhuang}@fudan.edu.cn\nAbstract\nLarge language models (LLMs) have a wealth\nof knowledge that allows them to excel in vari-\nous Natural Language Processing (NLP) tasks.\nCurrent research focuses on enhancing their\nperformance within their existing knowledge.\nDespite their vast knowledge, LLMs are still\nlimited by the amount of information they can\naccommodate and comprehend. Therefore, the\nability to understand their own limitations on\nthe unknows, referred to as self-knowledge,\nis of paramount importance. This study aims\nto evaluate LLMs’ self-knowledge by assess-\ning their ability to identify unanswerable or\nunknowable questions. We introduce an aut

In [23]:
doc_obj.page_content

'Do Large Language Models Know What They Don’t Know?\nZhangyue Yin♢Qiushi Sun♠Qipeng Guo♢\nJiawen Wu♢Xipeng Qiu♢∗Xuanjing Huang♢\n♢School of Computer Science, Fudan University\n♠Department of Mathematics, National University of Singapore\n{yinzy21,jwwu21}@m.fudan.edu.cn qiushisun@u.nus.edu\n{qpguo16,xpqiu,xjhuang}@fudan.edu.cn\nAbstract\nLarge language models (LLMs) have a wealth\nof knowledge that allows them to excel in vari-\nous Natural Language Processing (NLP) tasks.\nCurrent research focuses on enhancing their\nperformance within their existing knowledge.\nDespite their vast knowledge, LLMs are still\nlimited by the amount of information they can\naccommodate and comprehend. Therefore, the\nability to understand their own limitations on\nthe unknows, referred to as self-knowledge,\nis of paramount importance. This study aims\nto evaluate LLMs’ self-knowledge by assess-\ning their ability to identify unanswerable or\nunknowable questions. We introduce an auto-\nmated methodology 

In [24]:
from IPython.display import display, Markdown

Markdown(doc_obj.page_content)

Do Large Language Models Know What They Don’t Know?
Zhangyue Yin♢Qiushi Sun♠Qipeng Guo♢
Jiawen Wu♢Xipeng Qiu♢∗Xuanjing Huang♢
♢School of Computer Science, Fudan University
♠Department of Mathematics, National University of Singapore
{yinzy21,jwwu21}@m.fudan.edu.cn qiushisun@u.nus.edu
{qpguo16,xpqiu,xjhuang}@fudan.edu.cn
Abstract
Large language models (LLMs) have a wealth
of knowledge that allows them to excel in vari-
ous Natural Language Processing (NLP) tasks.
Current research focuses on enhancing their
performance within their existing knowledge.
Despite their vast knowledge, LLMs are still
limited by the amount of information they can
accommodate and comprehend. Therefore, the
ability to understand their own limitations on
the unknows, referred to as self-knowledge,
is of paramount importance. This study aims
to evaluate LLMs’ self-knowledge by assess-
ing their ability to identify unanswerable or
unknowable questions. We introduce an auto-
mated methodology to detect uncertainty in the
responses of these models, providing a novel
measure of their self-knowledge. We further in-
troduce a unique dataset, SelfAware , consisting
of unanswerable questions from five diverse cat-
egories and their answerable counterparts. Our
extensive analysis, involving 20 LLMs includ-
ing GPT-3, InstructGPT, and LLaMA, discov-
ering an intrinsic capacity for self-knowledge
within these models. Moreover, we demon-
strate that in-context learning and instruction
tuning can further enhance this self-knowledge.
Despite this promising insight, our findings also
highlight a considerable gap between the capa-
bilities of these models and human proficiency
in recognizing the limits of their knowledge.
“True wisdom is knowing what you don’t know.”
–Confucius
1 Introduction
Recently, Large Language Models (LLMs) such
as GPT-4 (OpenAI, 2023), PaLM 2 (Anil et al.,
2023), and LLaMA (Touvron et al., 2023) have
shown exceptional performance on a wide range
of NLP tasks, including common sense reason-
ing (Wei et al., 2022; Zhou et al., 2022) and mathe-
∗Corresponding author.
UnknowsKnows UnknowsKnows
Known KnowsKnown Unknows
Unknown Unknows Unknown KnowsUnlockFigure 1: Know-Unknow Quadrant. The horizontal axis
represents the model’s memory capacity for knowledge,
and the vertical axis represents the model’s ability to
comprehend and utilize knowledge.
matical problem-solving (Lewkowycz et al., 2022;
Chen et al., 2022). Despite their ability to learn
from huge amounts of data, LLMs still have lim-
itations in their capacity to retain and understand
information. To ensure responsible usage, it is cru-
cial for LLMs to have the capability of recognizing
their limitations and conveying uncertainty when
responding to unanswerable or unknowable ques-
tions. This acknowledgment of limitations, also
known as “ knowing what you don’t know ,” is a
crucial aspect in determining their practical appli-
cability. In this work, we refer to this ability as
model self-knowledge.
The Know-Unknow quadrant in Figure 1 il-
lustrates the relationship between the model’s
knowledge and comprehension. The ratio of
“Known Knows” to “Unknown Knows” demon-
strates the model’s proficiency in understanding
and applying existing knowledge. Techniques
such as Chain-of-Thought (Wei et al., 2022), Self-
Consistency (Wang et al., 2022), and Complex
CoT (Fu et al., 2022) can be utilized to increasearXiv:2305.18153v2  [cs.CL]  30 May 2023

In [25]:
embeddings = OpenAIEmbeddings() # EMBED
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x107f70090>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x11fe67a90>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base='https://api.openai.com/v1', openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None)

In [26]:
vectordb = Chroma.from_documents(pdf_docs, embedding=embeddings) # STORE
vectordb

In [27]:
retriever = vectordb.as_retriever() 
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x148c572d0>)

In [28]:
llm = ChatOpenAI(model="gpt-4-0125-preview")

In [29]:
llm.invoke("How can I be a better instructor for my live-trainings about AI to get everyone excited about LLMs?")

KeyboardInterrupt: 

In [34]:
pdf_qa = RetrievalQA.from_llm(llm=llm, retriever=retriever, return_source_documents=True) # RETRIEVE
pdf_qa
# pdf_qa = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=retriever) # RETRIEVE
# pdf_qa

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x175716990>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1756cbf50>, model_name='gpt-4-0125-preview', openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy='')), document_prompt=PromptTemplate(input_variables=['page_content'], template='Context:\n{page_content}'), document_variable_name='context'), re

In [13]:
query = "Summarize this paper into a set of instructive bullet points."

In [14]:
output = pdf_qa.invoke({"query": query, "chat_history": []}) # adding chat history so the model remembers previous questions

In [15]:
output

{'query': 'Summarize this paper into a set of instructive bullet points.',
 'chat_history': [],
 'result': "- The study evaluates the self-knowledge of Large Language Models (LLMs) such as GPT-3, InstructGPT, LLaMA, and their derivatives (Alpaca, Vicuna) by assessing their ability to identify unanswerable questions.\n- Utilized the SelfAware dataset, which includes a mix of unanswerable and answerable questions from various sources, to measure models' performance.\n- Developed an automated method for detecting uncertainty in model responses, using a set of reference sentences displaying uncertainty and computing similarity through SimCSE.\n- Conducted experiments with different input forms (Direct, Instruction, and In-Context Learning (ICL)) to see their impact on models' self-knowledge.\n- Found that models from the InstructGPT series and their instruction-tuned derivatives (Alpaca, Vicuna) showed improved self-knowledge over GPT-3.\n- ICL input form significantly enhanced models' sel

In [16]:
final_answer = output["result"]

Markdown(final_answer)

- The study evaluates the self-knowledge of Large Language Models (LLMs) such as GPT-3, InstructGPT, LLaMA, and their derivatives (Alpaca, Vicuna) by assessing their ability to identify unanswerable questions.
- Utilized the SelfAware dataset, which includes a mix of unanswerable and answerable questions from various sources, to measure models' performance.
- Developed an automated method for detecting uncertainty in model responses, using a set of reference sentences displaying uncertainty and computing similarity through SimCSE.
- Conducted experiments with different input forms (Direct, Instruction, and In-Context Learning (ICL)) to see their impact on models' self-knowledge.
- Found that models from the InstructGPT series and their instruction-tuned derivatives (Alpaca, Vicuna) showed improved self-knowledge over GPT-3.
- ICL input form significantly enhanced models' self-knowledge by providing richer contextual information.
- Compared model performance to human benchmarks, finding that while models like GPT-4 showed high F1 scores, they still lag behind human self-knowledge.
- Observed a steady improvement in the InstructGPT series' performance in answering answerable questions with increasing model size and continuous learning.
- Highlighted limitations including the generalization of reference sentences and the confinement to specific input forms; suggests future research directions like incorporating more cognitive and decision-making methods.
- Concluded that while LLMs demonstrate a degree of self-knowledge in identifying unanswerable questions, there's substantial room for enhancement to match human capabilities.
- Emphasized the importance of advancing LLMs' self-knowledge for more accurate and reliable applications across various domains.

In [35]:
query_summary = "What is the main discovery of the paper regarding self-knowledge in LLMs?"

output = pdf_qa.invoke({"query": query_summary, "chat_history": []}) # adding chat history so the model remembers previous questions

In [36]:
output

{'query': 'What is the main discovery of the paper regarding self-knowledge in LLMs?',
 'chat_history': [],
 'result': 'The main discovery of the paper regarding self-knowledge in Large Language Models (LLMs) is that these models inherently possess an intrinsic capacity for self-knowledge. This capacity is demonstrated through their ability to identify unanswerable or unknowable questions to some extent. The paper also finds that techniques such as in-context learning and instruction tuning can further enhance the self-knowledge of LLMs. Despite these promising insights, the study highlights a considerable gap between the capabilities of LLMs and human proficiency in recognizing the limits of their knowledge, with the state-of-the-art model, GPT-4, exhibiting self-knowledge at 75.47% compared to human self-knowledge rated at 84.93%.',
 'source_documents': [Document(page_content='Do Large Language Models Know What They Don’t Know?\nZhangyue Yin♢Qiushi Sun♠Qipeng Guo♢\nJiawen Wu♢Xipeng Q

[Long context issue in LLMs](https://arxiv.org/pdf/2303.18223.pdf)

Long Context. One of the main drawbacks of Transformerbased language models is the context length is limited due to the involved quadratic computational costs in both time
and memory. Meanwhile, there is an increasing demand
for LLM applications with long context windows, such as
in PDF processing and story writing [217]. ChatGPT has
recently released an updated variant with a context window
size of up to 16K tokens, which is much longer than the
initial one, i.e., 4K tokens. Additionally, GPT-4 was launched
with variants with context window of 32K tokens [46]. Next,
we discuss two important factors that support long context
modeling for LLMs.

# References

https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb 
Below are notebook from openai cookbook on these topics of search and embeddings:
- https://github.com/openai/openai-cookbook/blob/main/examples/Get_embeddings.ipynb
- https://github.com/openai/openai-cookbook/blob/main/examples/Code_search.ipynb
- https://github.com/openai/openai-cookbook/blob/main/examples/Customizing_embeddings.ipynb
- https://github.com/openai/openai-cookbook/blob/main/examples/Embedding_Wikipedia_articles_for_search.ipynb
- https://platform.openai.com/docs/guides/embeddings/what-are-embeddings
- [In-context learning abilities of ChatGPT models](https://arxiv.org/pdf/2303.18223.pdf)
- [Issue with long context](https://arxiv.org/pdf/2303.18223.pdf)